# compare LSE and SBE

In [1]:
import sys, os, glob, pickle, toml, json, pickle, random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.model_selection import GroupKFold,  KFold

def set_background(ax):
    ax.set_extent([-121, -72, 25, 50])

    # Background boundaries
    # ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    ax.add_feature(states_provinces, edgecolor='gray')
    
    return ax
    
def get_rmse(d1, d2):
    return ( np.nanmean( (d1-d2)**2 ) ) ** 0.5

def get_cc(d1, d2):
    ind = ~np.isnan(d1+d2)
    return np.corrcoef(d1[ind], d2[ind])[0,1]

def get_nse(obs, sim):     
    return 1-(np.nansum((sim-obs)**2)/np.nansum((obs-np.nanmean(obs))**2))

def get_mae(d1, d2):
    return np.nanmean( np.abs(d1 - d2) )

def get_modified_KGE(obs,sim):
    ind = (~np.isnan(obs)) & (~np.isnan(sim))
    obs = obs[ind]
    sim = sim[ind]

    try:
        sd_sim=np.std(sim, ddof=1)
        sd_obs=np.std(obs, ddof=1)
        m_sim=np.mean(sim)
        m_obs=np.mean(obs)
        r=(np.corrcoef(sim,obs))[0,1]
        relvar=(float(sd_sim)/float(m_sim))/(float(sd_obs)/float(m_obs))
        bias=float(m_sim)/float(m_obs)
        kge=1.0-np.sqrt((r-1)**2 +(relvar-1)**2 + (bias-1)**2)
    except:
        kge = np.nan

    return kge

def combine_cv_test(cv_result):
    dataplot = []
    dinput = np.array([])
    dest = np.array([])
    dindex = np.array([])

    # check if 0 is in
    if 0 in cv_result:
    
        for i in range(len(cv_result)):
            dinput = np.hstack([dinput, np.squeeze(cv_result[i]['y_test'])])
            dest = np.hstack([dest,  np.squeeze(cv_result[i]['y_test_pred'])])
            dindex = np.hstack([dindex,  np.squeeze(cv_result[i]['test_index'])])

    else:

        for i in range(1, len(cv_result)+1):
            dinput = np.hstack([dinput, np.squeeze(cv_result[i]['y_test'])])
            dest = np.hstack([dest,  np.squeeze(cv_result[i]['y_test_pred'])])
            dindex = np.hstack([dindex,  np.squeeze(cv_result[i]['test_index'])])
    
    dd = np.vstack([dindex, dinput, dest]).T
    return dd

# Load data and calculate metric for each basin

In [2]:
# load info
infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
df_info = pd.read_csv(infile_basin_info)

## SSE

In [3]:
# load data

path = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out'

methods1 = ['gpr', 'rf', 'mlp']
methods2 = ['GPR', 'RF' , 'MLP']
numbasin = 627

# cv_est_sse = []
# cv_eval_sse = np.nan * np.zeros([numbasin, 5, 2, 2, 3]) # numbasin; 5 fold; rmse/cc; train/test; gpr/rf/mlp

# for i in range(3):

#     file = f'{path}/SSE_{methods2[i]}_normKGE_CV_estimates.pkl'
#     with open(file, 'rb') as file:
#         cv_est_sse.append( pickle.load(file) )

#     file =  f'{path}/SSE_{methods2[i]}_normKGE_CV_evaluation.npz'
#     dtmp = np.load(file)
#     cv_eval_sse[:,:,0,0,i] = dtmp[f'{methods1[i]}_rmse_train']
#     cv_eval_sse[:,:,0,1,i] = dtmp[f'{methods1[i]}_rmse_test']
#     cv_eval_sse[:,:,1,0,i] = dtmp[f'{methods1[i]}_cc_train']
#     cv_eval_sse[:,:,1,1,i] = dtmp[f'{methods1[i]}_cc_test']

# cv_eval_sse = np.nanmean(cv_eval_sse, axis=1)


cv_eval_sse = np.nan * np.zeros([numbasin, 4, 3]) # numbasin; rmse/cc/mae/nse; gpr/rf/mlp

dtest1 = []
dtest2 = []
for i in range(3):

    file = f'{path}/SSE_{methods2[i]}_normKGE_CV_estimates.pkl'
    with open(file, 'rb') as file:
        cv_i = pickle.load(file)

    for j in range(numbasin):
        cv_ij = combine_cv_test( cv_i[j] )

        cv_eval_sse[j, 0, i] = get_rmse(cv_ij[:,1], cv_ij[:,2])
        cv_eval_sse[j, 1, i] = get_cc(cv_ij[:,1], cv_ij[:,2])
        cv_eval_sse[j, 2, i] = get_mae(cv_ij[:,1], cv_ij[:,2])
        cv_eval_sse[j, 3, i] = get_modified_KGE(cv_ij[:,1], cv_ij[:,2])

        dtest1.append(np.nanmean(cv_ij[:,1]))
        dtest2.append(get_mae(cv_ij[:,1], cv_ij[:,2]))
        
print(cv_eval_sse.shape)

(627, 4, 3)


## LSE

In [4]:
path = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out'

methods1 = ['rf', 'mlp']
methods2 = ['RF', 'MLP']
metprefix = [ ['rf', 'rf_group',  'rf_cluster'], 
             ['mlp', 'mlp_group',  'mlp_cluster']]
types = ['standard', 'group', 'cluster']

cv_est_lse = []

lab = []

for i in range(2):
    for j in range(3):
        lab.append(f'{methods2[i]} {types[j]}')
        
        file = f'{path}/LSE_{methods2[i]}_normKGE_{types[j]}CV_estimates.pkl'
        with open(file, 'rb') as file:
            cv_est_lse.append(pickle.load(file))
        
# cv_est_lse = np.array(cv_est_lse)
# print(cv_est_lse.shape)
print(lab)

['RF standard', 'RF group', 'RF cluster', 'MLP standard', 'MLP group', 'MLP cluster']


In [5]:
file = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out/basin_id_forinput.gz'
df_basinid = pd.read_csv(file, compression='gzip')

cv_eval_lse = np.nan * np.zeros([len(df_info), 4, len(cv_est_lse)]) # rmse/cc/mae/nse; ['RF standard', 'RF group', 'RF cluster', 'MLP standard', 'MLP group', 'MLP cluster']

for cc in range(len(cv_est_lse)):
    cvi = cv_est_lse[cc]
    
    dataplot = []
    dinput = np.array([])
    dest = np.array([])
    dindex = np.array([])
    for i in range(len(cvi)):
        dinput = np.hstack([dinput, np.squeeze(cvi[i]['y_test'])])
        dest = np.hstack([dest,  np.squeeze(cvi[i]['y_test_pred'])])
        dindex = np.hstack([dindex,  np.squeeze(cvi[i]['test_index'])])
    
    dd = np.vstack([dindex, dinput, dest]).T
    ind = np.argsort(dindex)
    dd = dd[ind, :]
    if len(dd) != (dd[-1,0]-dd[0,0]+1):
        print('Warning! error')
    
    
    for i in range(len(df_info)):
        indi = np.where(df_basinid['basin_id'].values==i)[0]
        ddi = dd[indi, :]
        cv_eval_lse[i, 0, cc] = get_rmse(ddi[:,1], ddi[:,2])
        cv_eval_lse[i, 1, cc] = get_cc(ddi[:,1], ddi[:,2])
        cv_eval_lse[i, 2, cc] = get_mae(ddi[:,1], ddi[:,2])
        cv_eval_lse[i, 3, cc] = get_modified_KGE(ddi[:,1], ddi[:,2])

## SBE

In [6]:
# load data

path = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out'

methods1 = ['rf', 'mlp']
methods2 = ['RF', 'MLP']
metprefix = [ ['rf', 'rf_group', ], 
             ['mlp', 'mlp_group', ]]
types = ['standard', 'group']


cv_est_sbe = []
lab = []
for i in range(2):
    for j in range(2):
        cv_est_ij = np.zeros([0, 4])
        lab.append(f'{methods2[i]} {types[j]}')
        for c in range(15):
            
            file = f'{path}/SBE_cluster{c}_{methods2[i]}_normKGE_{types[j]}CV_estimates.pkl'
            with open(file, 'rb') as file:
                cvc = combine_cv_test(pickle.load(file))
                cvc = np.hstack([np.ones([cvc.shape[0], 1]) * c, cvc])
                cv_est_ij = np.vstack([cv_est_ij, cvc])
        
        ind = np.argsort(cv_est_ij[:,1])
        cv_est_ij = cv_est_ij[ind, :]
        cv_est_sbe.append(cv_est_ij)

cv_est_sbe = np.array(cv_est_sbe)
print(lab)
print(cv_est_sbe.shape)

['RF standard', 'RF group', 'MLP standard', 'MLP group']
(4, 247491, 4)


In [7]:
# load cluster information
infile = "../../camels_cluster/Manuela_Brunner_2020/flood_cluster_memberships_CAMELS.txt"
df_cluster = pd.read_csv(infile)
df_cluster = df_cluster.rename(
    columns={"Camels_IDs": "hru_id", "flood_cluster": "clusters"}
)
df_cluster2 = pd.DataFrame()

for id in df_info["hru_id"].values:
    dfi = df_cluster.loc[df_cluster["hru_id"] == id]
    df_cluster2 = pd.concat([df_cluster2, dfi])

df_cluster2.sel_index = np.arange(len(df_cluster2))
df_cluster = df_cluster2
del df_cluster2
df_cluster["clusters"] = df_cluster["clusters"] - 1  # starting from 0

if np.any(df_info["hru_id"].values - df_cluster['hru_id'].values != 0):
    print("Mistmatch between basins and clusters")
else:
    print("basins and clusters match")

basins and clusters match


/glade/derecho/scratch/guoqiang/tmp/ipykernel_25910/3393541547.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_cluster2.sel_index = np.arange(len(df_cluster2))


In [8]:
file = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out/basin_id_forinput.gz'
df_basinid = pd.read_csv(file, compression='gzip')

clusters = np.nan * np.zeros(len(df_basinid))
for i in range(len(df_info)):
    indi = df_basinid['basin_id'].values == i
    clusters[indi] = df_cluster.iloc[i]['clusters']

In [9]:

def get_group_cv_indices(df_basinid, nfold=5, rndseed=1234567890):

    random.seed(rndseed)
    np.random.seed(rndseed)

    # Initialize GroupKFold
    group_kfold = GroupKFold(n_splits=nfold)

    # Dictionary to store train and test indices
    group_cv_indices = {}

    # Cross-validation process to save indices
    # does not depend on random seeds
    for fold, (train_index, test_index) in enumerate(group_kfold.split(df_basinid, groups=df_basinid['basin_num'])):
        
        train_basin = np.unique(df_basinid.iloc[train_index]['basin_id'].values)
        test_basin = np.unique(df_basinid.iloc[test_index]['basin_id'].values)

        group_cv_indices[fold] = {'train_index': train_index, 'test_index': test_index, 'train_index_raw': df_basinid.iloc[train_index].index, 'test_index_raw': df_basinid.iloc[test_index].index} 
    return group_cv_indices

# Get the indices
nfold = 5
group_cv_indices_all = []


for usecluster in range(15): # 0-14: 15 clusters
    ind2 = clusters == usecluster
    df_basinid_c = df_basinid[ind2]
    group_cv_indices = get_group_cv_indices(df_basinid_c, nfold)

    group_cv_indices_all.append(group_cv_indices)

In [10]:

cv_est_2 = cv_est_sbe.copy()
trueindex = np.nan * np.zeros(cv_est_2.shape[1])
for i in range(15):
    group_cv_indices = group_cv_indices_all[i]
    indi = cv_est_2[0,:,0]==i
    falseindex_i = cv_est_2[0,:,1][indi]
    trueindex_i = np.nan * np.ones(len(falseindex_i))
    for j in range(5):
        ind1 = group_cv_indices[j]['test_index']
        ind2 = np.array(group_cv_indices[j]['test_index_raw'])
        if np.any( np.any(falseindex_i[ind1] - ind1 !=0) ):
            print('Error warning!')
        trueindex_i[ind1] = ind2
    trueindex[indi] = trueindex_i

for i in range(cv_est_2.shape[0]):
    cv_est_2[i, :, 1] = trueindex

In [11]:
cv_eval_sbe = np.nan * np.zeros([len(df_info), 4, cv_est_2.shape[0]]) # rmse/cc/mae/nse; ['RF standard', 'RF group', 'MLP standard', 'MLP group']

for i in range(len(df_info)):
    indi = np.where(df_basinid['basin_id'].values==i)[0]
    for j in range(cv_est_2.shape[0]):
        ddi = cv_est_2[j, indi, 2]
        cv_eval_sbe[i, 0, j] = get_rmse(cv_est_2[j, indi, 2],  cv_est_2[j, indi, 3])
        cv_eval_sbe[i, 1, j] = get_cc(cv_est_2[j, indi, 2],  cv_est_2[j, indi, 3])
        cv_eval_sbe[i, 2, j] = get_mae(cv_est_2[j, indi, 2],  cv_est_2[j, indi, 3])
        cv_eval_sbe[i, 3, j] = get_modified_KGE(cv_est_2[j, indi, 2],  cv_est_2[j, indi, 3])

# Box plot of various emulators

In [13]:
# rmse and cc from cv

# for sse, just gpr
df1_1 = pd.DataFrame({
    'rmse': cv_eval_sse[:, 0, 0], # numbasin; rmse/cc; gpr/rf/mlp
    'cc': cv_eval_sse[:, 1, 0],
    'mae': cv_eval_sse[:, 2, 0], # numbasin; rmse/cc; gpr/rf/mlp
    'kge': cv_eval_sse[:, 3, 0],
    'Emulator': 'SSE: GPR',
    'CV': 'In-basin CV'
})


# for sse, combine GPR+RF
rmse_sse = np.zeros(cv_eval_sse.shape[0])
cc_sse = np.zeros(cv_eval_sse.shape[0])
mae_sse = np.zeros(cv_eval_sse.shape[0])
kge_sse = np.zeros(cv_eval_sse.shape[0])
induse_rf = cv_eval_sse[:, 0, 0] > cv_eval_sse[:, 0, 1]
rmse_sse[induse_rf] = cv_eval_sse[induse_rf, 0, 1]
rmse_sse[~induse_rf] = cv_eval_sse[~induse_rf, 0, 0]
cc_sse[induse_rf] = cv_eval_sse[induse_rf, 1, 1]
cc_sse[~induse_rf] = cv_eval_sse[~induse_rf, 1, 0]
mae_sse[induse_rf] = cv_eval_sse[induse_rf, 2, 1]
mae_sse[~induse_rf] = cv_eval_sse[~induse_rf, 2, 0]
kge_sse[induse_rf] = cv_eval_sse[induse_rf, 3, 1]
kge_sse[~induse_rf] = cv_eval_sse[~induse_rf, 3, 0]


df1_2 = pd.DataFrame({
    'rmse': rmse_sse, 
    'cc': cc_sse,
    'mae': mae_sse, 
    'kge': kge_sse,
    'Emulator': 'SSE: GPR+RF',
    'CV': 'In-basin CV'
})

# cv_eval_lse  # rmse/cc; ['RF standard', 'RF group', 'RF cluster', 'MLP standard', 'MLP group', 'MLP cluster']
df2 = pd.DataFrame({
    'rmse': cv_eval_lse[:, 0, 0],
    'cc': cv_eval_lse[:, 1, 0],
    'mae': cv_eval_lse[:, 2, 0],
    'kge': cv_eval_lse[:, 3, 0],
    'Emulator': 'LSE',
    'CV': 'In-basin CV'
})

df3 = pd.DataFrame({
    'rmse': cv_eval_lse[:, 0, 1],
    'cc': cv_eval_lse[:, 1, 1],
    'mae': cv_eval_lse[:, 2, 1],
    'kge': cv_eval_lse[:, 3, 1],
    'Emulator': 'LSE',
    'CV': 'Spatial CV'
})

df4 = pd.DataFrame({
    'rmse': cv_eval_lse[:, 0, 2],
    'cc': cv_eval_lse[:, 1, 2],
    'mae': cv_eval_lse[:, 2, 2],
    'kge': cv_eval_lse[:, 3, 2],
    'Emulator': 'LSE',
    'CV': 'Cluster CV'
})

# cv_eval_sbe # rmse/cc; ['RF standard', 'RF group', 'MLP standard', 'MLP group']
df5 = pd.DataFrame({
    'rmse': cv_eval_sbe[:, 0, 0],
    'cc': cv_eval_sbe[:, 1, 0],
    'mae': cv_eval_sbe[:, 2, 0],
    'kge': cv_eval_sbe[:, 3, 0],
    'Emulator': 'SBE',
    'CV': 'In-basin CV'
})

df6 = pd.DataFrame({
    'rmse': cv_eval_sbe[:, 0, 1],
    'cc': cv_eval_sbe[:, 1, 1],
    'mae': cv_eval_sbe[:, 2, 1],
    'kge': cv_eval_sbe[:, 3, 1],
    'Emulator': 'SBE',
    'CV': 'Spatial CV'
})

df_plot = pd.concat([df1_1, df1_2, df2, df3, df4, df5, df6])


In [14]:
df_plot2 = df_plot[df_plot['kge']>-1]

# Create the plot using seaborn
sns.set(style="whitegrid")

titles = ['(a) MAE', '(b) CC', "(c) KGE'"]
ys = ['mae', 'cc', 'kge']
ylabs = ['MAE', 'CC', "KGE'"]

fig = plt.figure(figsize=(8, 7))


for i in range(3):
    fig.add_subplot(3, 1, i+1)
    ax = sns.violinplot(data=df_plot2, x="CV", y=ys[i], hue="Emulator", palette="Set2", linewidth=0.4, inner='box', inner_kws={'box_width':4},density_norm='width')
    # Set the axis labels and title
    ax.set_xlabel("")
    ax.set_ylabel(ylabs[i])
    ax.set_title(titles[i])
    ax.set_xticks([0, 1.2, 2.1])
    
    # Adjust the legend
    if i == 0:
        plt.legend(loc='upper left')
    else:
        ax.legend().set_visible(False)

# Show the plot
plt.tight_layout()
# plt.savefig('Allemulator_CV_violinplot.png', dpi=600, bbox_inches='tight', pad_inches=0)
plt.show()

In [29]:

# # Create the plot using seaborn
# sns.set(style="whitegrid")

# titles = ['(a) MAE', '(b) CC']
# ys = ['mae', 'cc']
# ylabs = ['MAE', 'CC']

# fig = plt.figure(figsize=(8, 7))


# for i in range(2):
#     fig.add_subplot(2, 1, i+1)
#     ax = sns.violinplot(data=df_plot, x="CV", y=ys[i], hue="Emulator", palette="Set2", linewidth=0.4, inner='box', inner_kws={'box_width':4},density_norm='width')
#     # Set the axis labels and title
#     ax.set_xlabel("")
#     ax.set_ylabel(ylabs[i])
#     ax.set_title(titles[i])
#     ax.set_xticks([0, 1.2, 2.1])
    
#     # Adjust the legend
#     if i == 0:
#         plt.legend(loc='upper left')
#     else:
#         ax.legend().set_visible(False)

# # Show the plot
# plt.tight_layout()
# plt.savefig('Allemulator_CV_violinplot.png', dpi=600, bbox_inches='tight', pad_inches=0)
# plt.show()

# final figure

MAE SBE/LSE mean 0.1515682528564479 0.15487339919384557
MAE SBE/LSE median 0.14948794532766013 0.11915747688996735
CC SBE/LSE mean 0.6869030969959777 0.5675504911529358
MAE SBE/LSE median 0.7354149800309617 0.6736025773373769


In [32]:

# Create the plot using seaborn
sns.set(style="whitegrid")

titles = ['(a) MAE', '(b) CC']
ys = ['mae', 'cc']
ylabs = ['MAE', 'CC']

lon = df_info['lon_cen'].values
lat = df_info['lat_cen'].values
proj = ccrs.AlbersEqualArea(central_longitude=-100)
transf = ccrs.PlateCarree()

fig = plt.figure(figsize=(12, 7))

ax = fig.add_subplot(1, 2, 1, projection=proj)
ax = set_background(ax)
data1 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'SBE') ]['mae'].values
data2 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'LSE') ]['mae'].values
print('MAE SBE/LSE mean', np.nanmean(data1), np.nanmean(data2))
print('MAE SBE/LSE median', np.nanmedian(data1), np.nanmedian(data2))
p = ax.scatter(lon, lat, 15, data1-data2, cmap='jet', vmin=-0.1, vmax=0.1, transform=transf)
ax.set_title('SBE MAE - LSE MAE')
    
cax = plt.colorbar(p, ax=ax, location='bottom', shrink=1, label='MAE', pad=0.05)
cax.ax.yaxis.set_ticks_position('right')
cax.ax.yaxis.set_label_position('left')


ax = fig.add_subplot(1, 2, 2, projection=proj)
ax = set_background(ax)
data1 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'SBE') ]['cc'].values
data2 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'LSE') ]['cc'].values
print('CC SBE/LSE mean', np.nanmean(data1), np.nanmean(data2))
print('MAE SBE/LSE median', np.nanmedian(data1), np.nanmedian(data2))
p = ax.scatter(lon, lat, 15, data1-data2, cmap='jet', vmin=-0.1, vmax=0.1, transform=transf)
ax.set_title('SBE CC - LSE CC')
    
cax = plt.colorbar(p, ax=ax, location='bottom', shrink=1, label='CC', pad=0.05)
cax.ax.yaxis.set_ticks_position('right')
cax.ax.yaxis.set_label_position('left')



In [41]:
cc_comp_cluster = np.nan * np.zeros(len(df_cluster))
mae_comp_cluster = np.nan * np.zeros(len(df_cluster))

data1 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'SBE') ]['mae'].values
data2 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'LSE') ]['mae'].values
for c in range(15):
    indc = df_cluster['clusters'].values ==c
    mae_comp_cluster[indc] = np.sum(data1[indc]<data2[indc]) / np.sum(indc)


data1 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'SBE') ]['cc'].values
data2 = df_plot.loc[ (df_plot['CV'] == 'Spatial CV') & (df_plot['Emulator'] == 'LSE') ]['cc'].values
for c in range(15):
    indc = df_cluster['clusters'].values ==c
    cc_comp_cluster[indc] = np.sum(data1[indc]>data2[indc]) / np.sum(indc)



fig = plt.figure(figsize=(12, 7))

ax = fig.add_subplot(1, 2, 1, projection=proj)
ax = set_background(ax)
p = ax.scatter(lon, lat, 15, mae_comp_cluster, cmap='coolwarm', vmin=0, vmax=1, transform=transf)
ax.set_title('Ratio: SBE MAE < LSE MAE')
    
cax = plt.colorbar(p, ax=ax, location='bottom', shrink=1, label='', pad=0.05)
cax.ax.yaxis.set_ticks_position('right')
cax.ax.yaxis.set_label_position('left')


ax = fig.add_subplot(1, 2, 2, projection=proj)
ax = set_background(ax)
p = ax.scatter(lon, lat, 15, cc_comp_cluster, cmap='coolwarm', vmin=0, vmax=1, transform=transf)
ax.set_title('Ratio: SBE CC > LSE CC')
    
cax = plt.colorbar(p, ax=ax, location='bottom', shrink=1, label='', pad=0.05)
cax.ax.yaxis.set_ticks_position('right')
cax.ax.yaxis.set_label_position('left')
